In [199]:
import pandas as pd

In [200]:
import os
import sqlite3

In [204]:
df_fusion = pd.read_csv("/home/users/shubham.kumar/projects/qct_training_framework/notebooks/fusion_v5_qure25k_output.csv")
df_v3net = pd.read_csv("/home/users/shubham.kumar/projects/qct_training_framework/notebooks/v3net_qure25k_output.csv")

In [205]:
df_v3 = pd.read_csv("/home/users/shubham.kumar/projects/qct_training_framework/notebooks/blend_hemorrhages_v3_qure25k_output.csv")



In [206]:
df_fusion

,StudyUID,sdh_model_score_crop
0,ID_913c22e705,0.982836
1,ID_4c708b6c9d,0.969742
2,1.2.840.113837.4259240251.1578787859.0,0.990773
3,ID_f39e150c2e,0.997765
4,ID_9cbfc370b0,0.981767
...,...,...
41764,ID_a829a49e2a,0.984411
41765,1.2.840.113619.2.55.3.2831165736.637.150966233...,0.050575
41766,1.2.840.113619.2.55.3.2831165735.212.151110809...,0.975797
41767,ID_f8c80cd805,0.995136


In [208]:
from tqdm.auto import tqdm
import sqlite3
import pandas as pd
import os

path = "/home/users/shubham.kumar/projects/ICH_classification_segmentation/qct_training_framework/all_studies.sqlite"
con = sqlite3.connect(path)
df = pd.read_sql_query("SELECT * from nlp", con)

# Verify that result of SQL query is stored in the dataframe
con.close()
root_dir = "/data_nas3/processed/hct/Cache/all_imgs_cache.dcm"
df_test = df[df.Status == "qure25k"]

path_do_not_exist =[]
exception = []
for idx in tqdm(df_test.index) :
    try :
        row = df_test.loc[idx]
        filepath = row["Filepath"]
        req_path = os.path.join(root_dir, filepath[:-7])
        if os.path.exists(req_path):
            continue
        else:
            path_do_not_exist.append(req_path)
            exception.append(row["StudyUID"])
    except :
        exception.append(row["StudyUID"])

  0%|          | 0/80949 [00:00<?, ?it/s]

In [151]:
df_test = df_test[~df_test["StudyUID"].isin(exception)]

In [152]:
df_test = df_test[df_test["ANY"] != 100]

In [156]:
df_negative = df_test[df_test["ANY"]==0]

In [157]:
df_sdh = df_test[(df_test["ANY"]==1) & (df_test["SDH"] == 1)]

In [160]:
df_test_sdh = pd.concat([df_sdh , df_negative])

In [172]:
df_fusion_merged = pd.merge(df_test_sdh , df_fusion, how = "inner" , on = "StudyUID")

In [173]:
df_v3net_merged = pd.merge(df_test_sdh , df_v3net, how = "inner" , on = "StudyUID")

In [174]:
df_v3_merged =  pd.merge(df_test_sdh , df_v3, how = "inner" , on = "StudyUID")

In [175]:
len(df_fusion_merged)

34750

In [176]:
a = set(df_fusion_merged.StudyUID.values)
b = set(df_v3net_merged.StudyUID.values)
c = set(df_v3_merged.StudyUID.values)

In [177]:
req_uids_to_compare = list(a.intersection(b,c))

In [178]:
df_fusion_merged = df_fusion_merged[df_fusion_merged["StudyUID"].isin(req_uids_to_compare)]
df_v3_merged = df_v3_merged[df_v3_merged["StudyUID"].isin(req_uids_to_compare)]
df_v3net_merged = df_v3net_merged[df_v3net_merged["StudyUID"].isin(req_uids_to_compare)]

In [179]:
import numpy as np
from statsmodels.stats import proportion
from sklearn import metrics
import torch

def find_nearest(array, value):
    val = np.min(array[array >= value])
    idx = np.abs(array - val).argmin()
    return idx

def print_tpr_tnr(idx, fpr, tpr, thresh):
    print(
        "tpr, tnr = {:0.4f}, {:0.4f} at threshold {:0.4f}".format(
            tpr[idx], 1 - fpr[idx], thresh[idx]
        )
    )

def get_th(idx,fpr,tpr,thresh) :
    return round(tpr[idx],4), round(1 - fpr[idx],4), round(thresh[idx],4)

def auc_ci(Y, Y_pred):
    A = metrics.roc_auc_score(Y, Y_pred)
    m = np.sum(Y == 1)
    n = np.sum(Y == 0)
    assert m + n == Y_pred.size

    P_xxy = A / (2 - A)
    P_xyy = 2 * (A ** 2) / (1 + A)
    SD = (A * (1 - A) + (m - 1) * (P_xxy - A ** 2) + (n - 1) * (P_xyy - A ** 2)) / (
        m * n
    )
    SE = np.sqrt(SD)

    A1 = A - 1.96 * SE
    A2 = A + 1.96 * SE
    return A1, A2


def print_conf(Y, Y_pred, thresh=0.5, verbose=True):
    conf = metrics.confusion_matrix(Y, Y_pred >= thresh, labels=[0, 1])
    tn, fp, fn, tp = conf.flatten()

    if verbose:
        print("threshold:", thresh)

        print("predicted")
        print("   0    1")
        print(conf)

    tpr = tp / (tp + fn)
    tpr_interval = proportion.proportion_confint(tp, (tp + fn), method="beta")
    spc = tn / (tn + fp)
    spc_interval = proportion.proportion_confint(tn, (tn + fp), method="beta")
    ppv = tp / (tp + fp)
    ppv_interval = proportion.proportion_confint(tp, (tp + fp), method="beta")

    youden = tpr + spc - 1
    auc = metrics.roc_auc_score(Y, Y_pred)
    ap = metrics.average_precision_score(Y, Y_pred)

    acc = (tp + tn) / conf.sum()

    if verbose:
        print("tpr/sensitivity: {:0.4f} ({:0.4f} - {:0.4f})".format(tpr, *tpr_interval))
        print("tnr/specificity: {:0.4f} ({:0.4f} - {:0.4f})".format(spc, *spc_interval))
        print("ppv / precision: {:0.4f} ({:0.4f} - {:0.4f})".format(ppv, *ppv_interval))
        print("accuracy: {:0.4f}".format(acc))
        print("youden : {:0.4f}".format(youden))

        auc_low, auc_hi = auc_ci(Y, Y_pred)
        print("AUC: {:0.4f} ({:0.4f} - {:0.4f})".format(auc, auc_low, auc_hi))
        print("AP: {:0.4f}".format(ap))

    return tpr, spc, ppv


def print_measures(Y, Y_pred):

    Y_score = Y_pred
    fpr, tpr, thresh = metrics.roc_curve(Y, Y_score)
    tnr = 1 - fpr

    auc = metrics.roc_auc_score(Y, Y_score)
    ap = metrics.average_precision_score(Y, Y_score)
    auc_low, auc_hi = auc_ci(Y, Y_score)
    print("AUC: {:0.4f} ({:0.4f} - {:0.4f})".format(auc, auc_low, auc_hi))
    print("AP: {:0.4f}".format(ap))
    
    sens, spec, thres= get_th(find_nearest(tpr, 0.93), fpr, tpr,thresh)
        
    print()
    print("somepoints on the curve:")
    print_tpr_tnr(find_nearest(tpr, 0.95), fpr, tpr, thresh)
    print_tpr_tnr(find_nearest(tpr, 0.93), fpr, tpr, thresh)
    print_tpr_tnr(find_nearest(tpr, 0.9), fpr, tpr, thresh)
    print_tpr_tnr(find_nearest(tpr, 0.85), fpr, tpr, thresh)
    print_tpr_tnr(find_nearest(tpr, 0.8), fpr, tpr, thresh)
    print_tpr_tnr(find_nearest(tpr, 0.7), fpr, tpr, thresh)
    print_tpr_tnr(find_nearest(tpr, 0.6), fpr, tpr, thresh)
    print()
    print_tpr_tnr(find_nearest(tpr - tnr, 0), fpr, tpr, thresh)
    print()
    print_tpr_tnr(find_nearest(tnr, 0.95), fpr, tpr, thresh)
    print_tpr_tnr(find_nearest(tnr, 0.9), fpr, tpr, thresh)
    print_tpr_tnr(find_nearest(tnr, 0.85), fpr, tpr, thresh)
    print_tpr_tnr(find_nearest(tnr, 0.8), fpr, tpr, thresh)
    print_tpr_tnr(find_nearest(tnr, 0.7), fpr, tpr, thresh)
    print_tpr_tnr(find_nearest(tnr, 0.6), fpr, tpr, thresh)
    
    return {"AUC":auc, "sens":sens, "spec":spec,"thres":thres}

#     return auc , sens , spec , thres

def FN(Y, Y_pred):
    count = 0
    for i in range(len(Y)):
        if Y[i] == 1 and Y_pred[i] <= 0.2:
            count += 1
    return count

def FP(Y, Y_pred):
    count = 0
    for i in range(len(Y)):
        if Y[i] == 0 and 1 - Y_pred[i] <= 0.2:
            count += 1
    return count

def TP(Y, Y_pred):
    count = 0
    for i in range(len(Y)):
        if Y[i] == 1 and Y_pred[i] >= 0.8:
            count += 1
    return count

def TN(Y, Y_pred):
    count = 0
    for i in range(len(Y)):
        if Y[i] == 0 and 1 - Y_pred[i] >= 0.8:
            count += 1
    return count

def get_far_south_and_far_north_cases_count(Y, Y_pred):
    return TP(Y, Y_pred), TN(Y, Y_pred), FP(Y, Y_pred), FN(Y, Y_pred)



In [180]:
df_fusion_merged["ANY"].values

array([1, 1, 1, ..., 0, 0, 0])

In [203]:
df_fusion_merged

,StudyUID,PatientID,Status,Filepath,SeriesUID,Batch,Remarks,ICH,SDH,EDH,...,FRAC,CONFIRMED_HCT,POSTOP,NORMAL_BRAIN,UNCERTAIN_REPORT,ANY,crop,PATIENT_AGE,MANUFACTURER,sdh_model_score_crop
0,1.2.840.113619.2.22.288.1.8523.20181130.202036,HI-TECH HOSPITAL,qure25k,test_set_fts/sampled/1.2.840.113619.2.22.288.1...,1.2.840.113619.2.22.288.1.8523.2.20181130.202149,test_set_fts,nii.gz attempted,0,1,0,...,0,1,0,0,0.0,1,None,15.0,GE MEDICAL SYSTEMS,0.984876
1,1.2.840.113619.2.22.288.1.8530.20181201.302306,HI-TECH HOSPITAL,qure25k,test_set_fts/sampled/1.2.840.113619.2.22.288.1...,1.2.840.113619.2.22.288.1.8530.2.20181201.302350,test_set_fts,nii.gz attempted,0,1,0,...,1,1,0,1,0.0,1,None,8.0,GE MEDICAL SYSTEMS,0.949591
2,1.2.840.113619.2.81.290.1.14251.20180730.250058,010930072018,qure25k,test_set_fts/sampled/1.2.840.113619.2.81.290.1...,1.2.840.113619.2.81.290.1.14251.2.20180730.250140,test_set_fts,nii.gz attempted,0,1,0,...,1,1,0,0,0.0,1,None,40.0,GE MEDICAL SYSTEMS,0.976960
3,1.2.840.113619.2.81.290.1.14661.20181119.191432,047019112018,qure25k,test_set_fts/sampled/1.2.840.113619.2.81.290.1...,1.2.840.113619.2.81.290.1.14661.2.20181119.191524,test_set_fts,nii.gz attempted,0,1,1,...,1,1,0,0,0.0,1,None,28.0,GE MEDICAL SYSTEMS,0.973980
4,1.2.840.113619.2.81.290.1.14694.20181126.325919,050126112018,qure25k,test_set_fts/sampled/1.2.840.113619.2.81.290.1...,1.2.840.113619.2.81.290.1.14694.2.20181126.330022,test_set_fts,nii.gz attempted,0,1,0,...,1,1,0,0,0.0,1,None,25.0,GE MEDICAL SYSTEMS,0.852689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
34745,1.3.46.670589.33.1.63738846075211421100001.558...,MRN#12345,qure25k,test_set_vRad_6/sampled/1.3.46.670589.33.1.637...,1.3.46.670589.33.1.63738846510077294000001.552...,test_set_vRad_6,nii.gz attempted,0,0,0,...,0,1,0,0,0.0,0,None,80.0,Philips,0.974795
34746,1.3.46.670589.33.1.63739109503585153800001.556...,MRN#12345,qure25k,test_set_vRad_6/sampled/1.3.46.670589.33.1.637...,1.3.46.670589.33.1.63739109672765830400001.497...,test_set_vRad_6,nii.gz attempted,0,0,0,...,0,0,0,0,NaN,0,None,72.0,Philips,0.574204
34747,2.16.840.1.114151.1581139742814410481349717796...,MRN#12345,qure25k,test_set_vRad_6/sampled/2.16.840.1.114151.1581...,1.2.840.113619.2.358.3.4178195210.503.16031902...,test_set_vRad_6,nii.gz attempted,0,0,0,...,0,1,0,0,1.0,0,None,58.0,GE MEDICAL SYSTEMS,0.318282
34748,2.16.840.1.114151.1682282272467191296279541271...,MRN#12345,qure25k,test_set_vRad_6/sampled/2.16.840.1.114151.1682...,1.2.840.113619.2.358.3.4178195210.81.160310137...,test_set_vRad_6,nii.gz attempted,0,0,0,...,0,1,0,0,0.0,0,None,58.0,GE MEDICAL SYSTEMS,0.314030


In [181]:
df_fusion_merged["sdh_model_score_crop"].values

array([0.9848756 , 0.94959056, 0.9769603 , ..., 0.31828153, 0.3140305 ,
       0.46959674])

In [188]:
li = []

In [189]:
di = print_measures(df_fusion_merged["ANY"].values, df_fusion_merged["sdh_model_score_crop"].values)
di.update({"model":"fusionv5"})
li.append(di)

AUC: 0.9730 (0.9700 - 0.9761)
AP: 0.9041

somepoints on the curve:
tpr, tnr = 0.9501, 0.8681 at threshold 0.4537
tpr, tnr = 0.9300, 0.9223 at threshold 0.6409
tpr, tnr = 0.9001, 0.9525 at threshold 0.7752
tpr, tnr = 0.8504, 0.9726 at threshold 0.8748
tpr, tnr = 0.8000, 0.9820 at threshold 0.9198
tpr, tnr = 0.7001, 0.9902 at threshold 0.9553
tpr, tnr = 0.6004, 0.9936 at threshold 0.9712

tpr, tnr = 0.9266, 0.9266 at threshold 0.6574

tpr, tnr = 0.9048, 0.9501 at threshold 0.7612
tpr, tnr = 0.9396, 0.9004 at threshold 0.5562
tpr, tnr = 0.9537, 0.8501 at threshold 0.4063
tpr, tnr = 0.9678, 0.8002 at threshold 0.3015
tpr, tnr = 0.9810, 0.7012 at threshold 0.1754
tpr, tnr = 0.9853, 0.6043 at threshold 0.1138


In [190]:

di = print_measures(df_v3net_merged["ANY"].values, df_v3net_merged["sdh_model_score_crop"].values)
di.update({"model":"v3net"})
li.append(di)

AUC: 0.9746 (0.9716 - 0.9775)
AP: 0.9112

somepoints on the curve:
tpr, tnr = 0.9501, 0.8936 at threshold 0.4340
tpr, tnr = 0.9300, 0.9325 at threshold 0.5862
tpr, tnr = 0.9001, 0.9569 at threshold 0.7233
tpr, tnr = 0.8500, 0.9759 at threshold 0.8477
tpr, tnr = 0.8002, 0.9836 at threshold 0.8986
tpr, tnr = 0.7001, 0.9913 at threshold 0.9452
tpr, tnr = 0.6008, 0.9943 at threshold 0.9643

tpr, tnr = 0.9308, 0.9307 at threshold 0.5786

tpr, tnr = 0.9133, 0.9503 at threshold 0.6816
tpr, tnr = 0.9460, 0.9004 at threshold 0.4583
tpr, tnr = 0.9597, 0.8503 at threshold 0.3317
tpr, tnr = 0.9695, 0.8034 at threshold 0.2562
tpr, tnr = 0.9801, 0.7014 at threshold 0.1604
tpr, tnr = 0.9851, 0.6056 at threshold 0.1128


In [191]:
di = print_measures(df_v3_merged["ANY"].values, df_v3_merged["hemorrhages_v3"].values)
di.update({"model":"hemorrhages_v3"})
li.append(di)

AUC: 0.9757 (0.9728 - 0.9787)
AP: 0.9179

somepoints on the curve:
tpr, tnr = 0.9501, 0.8996 at threshold 0.5205
tpr, tnr = 0.9300, 0.9391 at threshold 0.6657
tpr, tnr = 0.9001, 0.9638 at threshold 0.7876
tpr, tnr = 0.8500, 0.9796 at threshold 0.8847
tpr, tnr = 0.8004, 0.9862 at threshold 0.9267
tpr, tnr = 0.7001, 0.9918 at threshold 0.9630
tpr, tnr = 0.6015, 0.9948 at threshold 0.9790

tpr, tnr = 0.9343, 0.9343 at threshold 0.6460

tpr, tnr = 0.9178, 0.9509 at threshold 0.7221
tpr, tnr = 0.9498, 0.9009 at threshold 0.5261
tpr, tnr = 0.9614, 0.8513 at threshold 0.3981
tpr, tnr = 0.9690, 0.8006 at threshold 0.3045
tpr, tnr = 0.9793, 0.7054 at threshold 0.1922
tpr, tnr = 0.9848, 0.6034 at threshold 0.1203


In [192]:
di = print_measures(df_v3_merged["ANY"].values, df_v3_merged["blend_score"].values)
di.update({"model":"blend"})
li.append(di)

AUC: 0.9730 (0.9699 - 0.9761)
AP: 0.9022

somepoints on the curve:
tpr, tnr = 0.9501, 0.8880 at threshold 0.5155
tpr, tnr = 0.9300, 0.9257 at threshold 0.6091
tpr, tnr = 0.9001, 0.9507 at threshold 0.6962
tpr, tnr = 0.8500, 0.9707 at threshold 0.7887
tpr, tnr = 0.8002, 0.9794 at threshold 0.8505
tpr, tnr = 0.7003, 0.9888 at threshold 0.9218
tpr, tnr = 0.6019, 0.9937 at threshold 0.9581

tpr, tnr = 0.9278, 0.9277 at threshold 0.6153

tpr, tnr = 0.9003, 0.9500 at threshold 0.6948
tpr, tnr = 0.9445, 0.9006 at threshold 0.5451
tpr, tnr = 0.9569, 0.8517 at threshold 0.4468
tpr, tnr = 0.9673, 0.8029 at threshold 0.3800
tpr, tnr = 0.9791, 0.7028 at threshold 0.2819
tpr, tnr = 0.9859, 0.6008 at threshold 0.2130


In [196]:
pd.DataFrame.from_records(li).to_csv("inference_table_sdh.csv",index =False)

'/home/users/shubham.kumar/projects/qct_training_framework/notebooks'

In [198]:
pd.DataFrame.from_records(li)

,AUC,sens,spec,thres,model
0,0.973038,0.93,0.9223,0.6409,fusionv5
1,0.974553,0.93,0.9325,0.5862,v3net
2,0.975744,0.93,0.9391,0.6657,hemorrhages_v3
3,0.972990,0.93,0.9257,0.6091,blend
